In [ ]:
from medigraph.data.abide import AbideData
import numpy as np
from nilearn import plotting
from medigraph.model.gcn import GCN
from medigraph.model.baseline import DenseNN
import torch
from tqdm.notebook import tqdm
from medigraph.data.preprocess import sanitize_data, visual_sanity_check_input, whiten
from medigraph.train import training_loop
from medigraph.data.properties import INPUTS, LABELS
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%load_ext autoreload
%autoreload 2
device

In [ ]:
dat = AbideData()

### Check connectivy matrix for a single patient
- 111x111 matrices
- We'll retrieve the $6216=\frac{111*(111+1)}{2}$ raw coefficients from the upper triangular matrix

In [ ]:
# Get the connectivity matrix for the first subject
idx = 0
mat = dat.get_connectivity_matrix(idx)
plotting.plot_matrix(
    mat,
    figure=(6, 6),
    vmax=1,
    vmin=0,
    title=f"Patient {idx} connectivity matrix {mat.shape}"
)
feature_vector_input = dat.get_connectivity_features(idx)
print(f"input feature vector shape: {feature_vector_input.shape}")

# Train classifier
### Build adjacency, features matrix and classification labels

In [ ]:
# % Build adjacency matrix and input feature vectors
inp_np, lab_np, adj_np = dat.get_training_data()
print(f"Adjacency matrix : {adj_np.shape} [VxV]")
print(f"Labels {lab_np.shape} : [V]")
print(f"Input feature vector {inp_np.shape} : [VxF]")

In [ ]:
# % Load data to GPU
labels_np = dat.get_labels()
adj = torch.tensor(adj_np, dtype=torch.float32).to(device)
inp_raw = torch.tensor(inp_np, dtype=torch.float32).to(device)  # [V=871,  F6216]
lab = torch.tensor(labels_np, dtype=torch.float32).to(device)  # for binary classification

In [ ]:
# % Sanitize and whiten data
clean_inp = sanitize_data(inp_raw)
inp = whiten(clean_inp)
inp.shape, adj.shape, lab.shape

In [ ]:
# % Visalization of sanity check
visual_sanity_check_input(inp_raw)
visual_sanity_check_input(clean_inp)
visual_sanity_check_input(inp)

In [ ]:
# % sanity check on graph adjacency matrix
model = GCN(inp.shape[1], adj, hdim=64)
plotting.plot_matrix(
    model.adj.detach().cpu().numpy(),
    figure=(6, 6),
    vmax=0.005,
    vmin=0,
    title=f"Graph normalized adjacency matrix {mat.shape}"
)
del model

In [ ]:
training_data = {
    INPUTS: inp,
    LABELS: lab
}
metric_dict = {}
for model_name in ["Dense", "GCN"]:
    if model_name == "GCN":
        model = GCN(inp.shape[1], adj, hdim=64)
    else:
        model = DenseNN(inp.shape[1], hdim=64)
    model.to(device)

    model, metrics = training_loop(model, training_data, lab, n_epochs=1000)
    metric_dict[model_name] = metrics

In [ ]:
def plot_metrics(metric_dict: dict):
    fig, axs = plt.subplots(1, 2, figsize=(10, 6))
    for model_name, metric in metric_dict.items():
        print(metric.keys())
        axs[0].plot(metric["training_losses"], label=model_name)
        axs[1].plot(metric["training_accuracies"], label=f"{model_name} accuracy")
    for ax in axs:
        ax.legend()
        ax.grid()
    axs[0].set_title("Training loss (Binary Cross Entropy)")
    axs[1].set_title("Accuracy")

    plt.show()


plot_metrics(metric_dict)